# Democratization Aid Visualizations

### Figure 1(a) : Total Dem Vs. Total ODA Trend

In [ ]:
#Import the Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


# Load first Excel file (Adjust the file path)
xls = pd.ExcelFile(r"D:\RA_IDOS\Julia Leninger\Recipients_2023.xlsx")
df = xls.parse("Recipient + year")

# Load second Excel file (Same file path)
xls2 = pd.ExcelFile(r"D:\RA_IDOS\Julia Leninger\Demo aid + 'ERT DS 2024 JL.xlsx")
df2 = xls2.parse("Sheet1")  

# Filter for years between 1995 and 2022
dem_aid_trend = df[(df["year"] >= 1995) & (df["year"] <= 2022)].groupby("year")["totalCommitted"].sum()
oda_trend = df2[(df2["year"] >= 1995) & (df2["year"] <= 2022)].groupby("year")["ODA"].sum()  # Adjust column name if needed

# Normalize values
dem_aid_trend = dem_aid_trend / 100  
oda_trend = oda_trend / 100  

# Plot both ODA and Democracy Aid
plt.figure(figsize=(10, 6))

# Plot Democracy Aid (dots)
plt.plot(dem_aid_trend.index, dem_aid_trend.values, marker='o', linestyle='-', color='black', label="Democracy Aid")

# Plot ODA (crosses)
plt.plot(oda_trend.index, oda_trend.values, marker='x', linestyle='--', color='black', label="Total ODA")

# Labels and Title
plt.xlabel("Year")
plt.ylabel("Amount (100 Million USD)")
plt.title("Total democracy aid and total Official Development Assistance, ODA (1995-2022)")

# Adding grid and legend
plt.grid(True, linestyle='--', alpha=0.6)

#Show all years
plt.xticks(range(1995, 2023), rotation=45)  

plt.legend()

# Save and Show the Plot
plt.savefig("plot.png", dpi=300) 
plt.show()


### Figure 1(b): Dem Aid with(out) USA

In [ ]:
# Load the data (Adjust the file path)
xls = pd.ExcelFile(r"D:\RA_IDOS\Julia Leninger\Recipients_2023.xlsx")
df = xls.parse("Recipient + year")
dem_aid_trend = df[(df["year"] >= 1995) & (df["year"] <= 2022)].groupby("year")["totalCommitted"].sum()  # Changed 2023 to 2022
dem_aid_trend = dem_aid_trend / 100  

# Load the donors' data
xls_donors = pd.ExcelFile(r"D:\RA_IDOS\Julia Leninger\Donors_2023.xlsx")
df_donors = xls_donors.parse("Sheet1")

# Find the total donation per year for all donors
total_donations = df_donors[(df_donors["year"] >= 1995) & (df_donors["year"] <= 2022)].groupby("year")["totalCommitted"].sum()  # Changed 2023 to 2022
total_donations = total_donations / 100  

# Find the USA donations per year
usa_donations = df_donors[(df_donors["donor_name"] == "United States") & (df_donors["year"] >= 1995) & (df_donors["year"] <= 2022)].groupby("year")["totalCommitted"].sum()  # Changed 2023 to 2022
usa_donations = usa_donations / 100  # In 100 Million USD

#USA donations - the total donations 
donations_without_usa = total_donations - usa_donations


plt.figure(figsize=(10, 6))
plt.plot(total_donations.index, total_donations.values, marker='x', linestyle='-', color='black', label="with USA")
plt.plot(donations_without_usa.index, donations_without_usa.values, marker='o', linestyle='-', color='black', label="without USA")

# Customize the plot
plt.xlabel("Year")
plt.ylabel("Total Donations (100 Million USD)")
plt.title("Total democracy aid with and without USA (1995-2022)")
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(ticks=total_donations.index, labels=total_donations.index, rotation=45)
plt.legend()
plt.savefig("donations_comparison_updated.png", dpi=300)  # Save as PNG with 300 dpi resolution
plt.show()

### Figure 2(a): Top Recipients(Adjusted)

In [ ]:
# Load dataset (Adjust the file path)
xls = pd.ExcelFile(r"D:\RA_IDOS\Julia Leninger\Demo aid + 'ERT DS 2024 JL.xlsx")
df = xls.parse("Sheet1")

# Define time periods
time_periods = {
    "1995-2001": (1995, 2001),
    "2002-2012": (2002, 2012),
    "2013-2023": (2013, 2023)
}

# Define shades of grey for top 10 in a given period
grey_shades = ['#B0B0B0', '#8C8C8C', '#666666']  

# Identify all-time top 10 countries across all periods
all_top_countries = set()
top_countries_per_period = {}

for period_name, (start_year, end_year) in time_periods.items():
    df_period = df[(df["year"] >= start_year) & (df["year"] <= end_year)]
    country_total_aid = df_period.groupby("country_name")["dem_aid"].sum()
    top_10_countries = country_total_aid.sort_values(ascending=False).head(10).index.tolist()
    top_countries_per_period[period_name] = top_10_countries
    all_top_countries.update(top_10_countries)

all_top_countries = sorted(all_top_countries)  # Sort alphabetically 

# Create subplots for visualizing top 10 countries in each period
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

for idx, (period_name, (start_year, end_year)) in enumerate(time_periods.items()):
    df_period = df[(df["year"] >= start_year) & (df["year"] <= end_year)]
    country_total_aid = df_period.groupby("country_name")["dem_aid"].sum()
    country_total_aid = country_total_aid.reindex(all_top_countries, fill_value=0)
    
    aid_values = country_total_aid / 100
    
    colors = [
        grey_shades[idx] if country in top_countries_per_period[period_name] else 'white'
        for country in all_top_countries
    ]
    
    # Plot bars
    bars = axes[idx].barh(all_top_countries[::-1], aid_values[::-1], 
                          color=colors[::-1], edgecolor='black')

    # Title and labels
    axes[idx].set_title(f"Top recipients of democracy aid ({period_name})")
    axes[idx].set_xlabel("Democracy Aid (100 Million USD)")
    axes[idx].grid(axis='x', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.savefig("Top_Recipients.png", dpi=300) 
plt.show()

### Figure 2(b): Top Recipients (Per Capita)(Adjusted)

In [ ]:
time_periods = {
    "1995-2001": (1995, 2001),
    "2002-2012": (2002, 2012),
    "2013-2023": (2013, 2023)
}

grey_shades = ['#B0B0B0', '#8C8C8C', '#666666']

selected_countries = set()
top_countries_pc_per_period = {}

for period_name, (start_year, end_year) in time_periods.items():
    df_period = df[(df["year"] >= start_year) & (df["year"] <= end_year)]
    country_total_aid_pc = df_period.groupby("country_name")["aid_pc"].sum()
    
    top_10_countries_pc = country_total_aid_pc.sort_values(ascending=False).head(10).index.tolist()
    top_countries_pc_per_period[period_name] = top_10_countries_pc
    selected_countries.update(top_10_countries_pc)

selected_countries = sorted(selected_countries)

fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

for idx, (period_name, (start_year, end_year)) in enumerate(time_periods.items()):
    df_period = df[(df["year"] >= start_year) & (df["year"] <= end_year)]
    country_total_aid_pc = df_period.groupby("country_name")["aid_pc"].sum()
    
  
    country_total_aid_pc = country_total_aid_pc.reindex(selected_countries, fill_value=0)
    

    aid_values_pc = country_total_aid_pc / 100
    

    colors = [
        grey_shades[idx] if country in top_countries_pc_per_period[period_name] else 'white'
        for country in selected_countries
    ]
    

    axes[idx].barh(selected_countries[::-1], aid_values_pc[::-1], 
                   color=colors[::-1], edgecolor='black')


    axes[idx].set_title(f"Top recipients of democracy aid per capita ({period_name})")
    axes[idx].set_xlabel("Democracy Aid per capita (100 Million USD)")
    axes[idx].grid(axis='x', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.savefig("Top_Recipients_Per_Capita.png", dpi=300)
plt.show()


### Figure 3: Top Donors

In [ ]:
xls = pd.ExcelFile( r"D:\RA_IDOS\Julia Leninger\Demo aid + 'ERT DS 2024 JL.xlsx") #Adjust the file path
df_d = xls.parse("Sheet1")

country_total_don = df_d.groupby("donor_name")["totalCommitted"].sum()
top_donors = country_total_don.sort_values(ascending= False).head(10)
top_donors= top_donors/100

plt.figure(figsize=(10, 6))
top_donors.plot(kind='bar', color='grey')
plt.title("Top Ten Donors of Democracy Aid (Total Amount, 1995-2023)")
plt.xlabel("Top 10 Donors")
plt.ylabel("Total Democracy Aid (100 Million USD)")
plt.xticks(rotation=45, ha="right")  
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout() 
plt.show()

### Figure 4(a): Dem Aid Vs. Aut Aid

In [ ]:
# Load the Excel file (Adjust the file path)
file_path = r"D:\RA_IDOS\Julia Leninger\Demo aid + 'ERT DS 2024 JL.xlsx" 
xls = pd.ExcelFile(file_path)
df = xls.parse("Sheet1")

# Filter out rows where important columns are missing or invalid
df_clean = df.dropna(subset=["year", "reg_type", "dem_aid"])

# Filter for years
df_clean = df_clean[(df_clean["year"] >= 1995) & (df_clean["year"] <= 2022)]

# Group by regime type (Autocracy = 0, Democracy = 1) and year, summing the aid per year
aid_by_regime = df_clean.groupby(["reg_type", "year"])["dem_aid"].sum().reset_index()

# Pivot the data to have separate columns for each regime type
aid_pivot = aid_by_regime.pivot(index="year", columns="reg_type", values="dem_aid").fillna(0)

# Rename columns
aid_pivot.columns = ["Autocracy", "Democracy"]
years = aid_pivot.index

width = 0.4  # Bar width

plt.figure(figsize=(12, 6))

#Autocracy bars (left)
plt.bar(years - width/2, aid_pivot["Autocracy"] / 100, width=width, color='black', label="Autocratization episodes", alpha=0.7)

#Democracy bars (right)
plt.bar(years + width/2, aid_pivot["Democracy"] / 100, width=width, color='gray', label="Democratization episodes", alpha=0.7)

# Formatting
plt.xlabel("Year")
plt.ylabel("Democracy Aid (100M USD)")
plt.title("Global Democracy Aid During Autocracy vs. Democracy")
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.xticks(rotation=45)
plt.xticks(ticks=years, labels=years, rotation=45)

plt.tight_layout()
plt.show()

### Figure 4(b): Dem Aid Vs. Aut Aid with lag

In [ ]:

aid_pivot_lagged_1 = aid_pivot.shift(-1) # Adjust the number

# Plot
plt.figure(figsize=(12, 6))
plt.bar(years - width/2, aid_pivot_lagged_1["Autocracy"] / 100, width=width, color='black', label="Lagged 1 year Autocratization", alpha=0.7)
plt.bar(years + width/2, aid_pivot_lagged_1["Democracy"] / 100, width=width, color='gray', label="Lagged 1 year Democratization", alpha=0.7)

# Formatting
plt.xlabel("Year")
plt.ylabel("Democracy Aid (100M USD)")
plt.title("Global Democracy Aid During Autocracy vs. Democracy (Lagged 1 Year)") #Adjust name
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.xticks(rotation=45)
plt.xticks(ticks=years, labels=years, rotation=45)
plt.tight_layo